**Import librariest**

In [1]:
import pandas as pd
import re

**Import dataset**

In [2]:
file_path = "final_merged_data_breaches.csv"
final_merged_data = pd.read_csv(file_path)

**Organization**

In [3]:
final_merged_data['organization'] = final_merged_data['organization'].replace('?', 'unknown').replace('various', 'unknown')


unification_dict = {
    "adobe": ["adobe inc.", "adobe systems incorporated"], "aerserv": ["aerserv (subsidiary of inmobi)"], "anthem": ["anthem inc."], "apple": ["apple health medicaid", "apple, inc./bluetoad", "apple icloud"],
    "ashley madison": ["ashleymadison.com"], "amazon": ["amazon japan g.k.", "amazon reviews"], "bell canada": ["bell"], "blue cross blue shield of tennessee": ["bluecross blueshield of tennessee"],
    "cardsystems solutions inc.": ["cardsystems solutions inc.(mastercard, visa, discover financial services and american express)"], "carefirst": ["carefirst bluecross blue shield - maryland"],
    "medicaid": ["centers for medicare & medicaid services", "medicare & medicaid"], "chinese gaming sites": ["chinese gaming websites (three: duowan, 7k7k, 178.com)"], "blizzard":["blizzard entertainment"],
    "colorado government": ["colorado department of health care policy & financing"], "desjardins": ["desjardins group"], "domino's pizza (france)": ["dominios pizzas (france)"], "gap inc": ["gap inc."],
    "educational credit management corp": ["educational credit management corporation"], "experian": ["experian - t-mobile us", "experian / t-mobile", "experian brazil", "experian sa"], "google plus": ["google+"],
    "faceboook": ["faceboook marketplace"], "gawker": ["gawker.com"], "health net": ["health net - ibm", "health net — ibm"], "ticketfly": ["ticketfly (subsidiary of eventbrite)"], "health net": ["health net — ibm"],
    "heartland": ["heartland payment systems"], "linkedin": ["linkedin, eharmony, last.fm"], "macrumors": ["macrumors.com"], "marriott": ["marriott hotels", "marriott international"], "microsoft": ["microsoft exchange servers"],
    "myfitnesspal": ["myfitnesspal (under armour subsidiary)"], "nintendo": ["nintendo (club nintendo)", "nintendo (nintendo account)"], "red cross": ["red cross blood service"], "massachusetts government": ["massachusetts executive office of labor and workforce development"],
    "saks": ["saks and lord & taylor", "saks fifth avenue / lord & taylor"], "shanghai": ["shanghai national police database", "shanghai police"],
    "sony": ["sony online entertainment", "sony pictures", "sony playstation network", "sony psn"], "t-mobile": ["t-mobile, deutsche telecom", "t-mobile, deutsche telekom"], "target": ["target corporation"], "tianya": ["tianya club"], 
    "ucla": ["ucla health", "ucla medical center, santa monica"], "university of wisconsin": ["university of wisconsin - milwaukee", "university of wisconsin–milwaukee"],
    "u.s census bureau": ["unknown agency(believed to be tied to united states census bureau)"], "us office of personnel management": ["us office of personnel management (2nd breach)"],
    "law enforcement agencies": ["various law enforcement agencies (philippine national police, national bureau of investigation, bureau of internal revenue)"],
    "verifications.io": ["verifications.io (first leak)", "verifications.io (total leaks)"], "volkswagen": ["vw"], "wawa": ["wawa (company)"], "yahoo": ["yahoo japan", "yahoo voices", "yahoo! voices"]
}

inverse_dict = {name: key for key, names in unification_dict.items() for name in names}
final_merged_data['organization'] = final_merged_data['organization'].replace(inverse_dict)

final_merged_data['organization'] = final_merged_data['organization'].str.replace('"', '', regex=False).str.strip().str.lower()

**Records lost**

In [4]:
def replace_textual_values(value):
    if isinstance(value, str):
        if re.search(r'\b(?:tb|gb|terabytes?|gigabytes?|years?|leaders?)\b', value, flags=re.IGNORECASE):
            return 'unknown'
        
        value = re.sub(r'[^\w\s,.]', '', value)
        
        match = re.search(r'(\d{1,3}(?:[,\.]?\d{3})*)', value)
        
        if match:
            cleaned_number = re.sub(r'[,.]', '', match.group(1))
            return cleaned_number
        return 'unknown'
    return 'unknown'

final_merged_data['records lost'] = final_merged_data['records lost'].apply(replace_textual_values)

# valore da rimuovere
final_merged_data.loc[(final_merged_data['organization'].str.lower() == 'heathrow airport') & (final_merged_data['records lost'] == '2'), 'records lost'] = 'unknown'


**Year**

In [5]:
def is_numeric(value):
    if isinstance(value, str):
        return bool(re.match(r'^\d+$', value))
    return isinstance(value, int)

cleaned_data = final_merged_data[final_merged_data['year'].apply(is_numeric) | final_merged_data['year'].apply(lambda x: isinstance(x, int))].copy()


**Sector**

In [6]:
def standardize_sector(sector):
    if isinstance(sector, str):
        sector = sector.lower().strip()
        sector_categories = {
            'technology and telecommunications': ['app', 'consumer genetics', 'background check', 'social network', 'genealogy', 'security', 'consumer goods', 'dating', 'educational services' 'electronics', 'hosting provider', 'information security', 
            'information technology', 'local search', 'malware tools', 'network monitoring', 'online marketing', 'software', 'tech', 'tech, cloud storage', 'tech, data storage', 'tech, health', 'tech, retail', 'tech, web', 'telecom', 'telecommunications', 'telecoms', 
            'telephone directory', 'misc, health', 'web', 'web services', 'web, gaming', 'web, military', 'web, tech'],
            'finance and insurance': ['broker/dealer', 'bureau', 'data broker', 'finance', 'financial', 'financial, credit bureau', 'financial, credit reporting', 'nonprofit, financial'],
            'retail and e-commerce': ['tech, app', 'online', 'market analysis', 'e-commerce', 'food', 'phone accessories', 'mobile carrier', 'hotel/casino', 'logistics', 'qr code payment', 'restaurant', 'automotives', 'retail', 'shopping', 'ticket distribution'],
            'healthcare': ['clinical laboratory', 'data aggregator', 'health', 'healthcare', 'healthcare, pharmacy', 'healthcare, retail', 'military, health', 'military, healthcare', 'government, health', 'government, healthcare', 'pharmacy'],
            'government and military': ['aerospace', 'government', 'personal and demographic data about residents and their properties of us', 'government, military', 'law enforcement', 'military', 'military, health', 'military, healthcare', 'political', 'politics', 'special public corporation'],
            'media and entertainment': ['advertising', 'arts group', 'entertainment', 'gambling', 'gaming', 'media', 'public broadcasting', 'publisher (magazine)', 'sports', 'web, gaming'],
            'academic institutions': ['academic'],
            'consumer and utility services': ['logistics', 'energy', 'local transport authority', 'printing', 'legal', 'transport', 'consulting, accounting'],
            'charity and non-profit': ['charity', 'ngo', 'humanitarian', 'computer services for charities'],
            'unknown': ['misc', 'international', 'various']
        }

        for category, terms in sector_categories.items():
            if any(term in sector for term in terms):
                return category
    return 'unknown'

cleaned_data['sector'] = cleaned_data['sector'].apply(standardize_sector)

def rename_unknown_sectors(data): #sostituisco gli unknown rimasti mettendoli nella giusta categoria
    replacements = {
        'cgi group': 'technology and telecommunications', 'ernst & young': 'consumer and utility services', 'collection no. 1': 'technology and telecommunications', '50 companies and government institutions': 'government and military', 
        'central hudson gas & electric': 'consumer and utility services', 'chtrbox': 'technology and telecommunications', 'gs caltex': 'consumer and utility services', 'mossack fonseca': 'finance and insurance', 'new york state electric & gas': 'consumer and utility services',
        'sony': 'technology and telecommunications', 'travelio': 'consumer and utility services', 'washington post': 'media and entertainment', 'world check': 'government and military', 'zhenhua': 'consumer and utility services',
        'quora': 'technology and telecommunications', 'australian red cross blood service': 'charity and non-profit', 'dell': 'technology and telecommunications', 'duolingo': 'technology and telecommunications', 'accenture': 'consumer and utility services', 'deloitte': 'consumer and utility services',
        'commission on elections': 'government and military', 'dai nippon printing': 'consumer and utility services', 'uber': 'technology and telecommunications', 'cathay pacific airways': 'consumer and utility services', 'japan airlines': 'consumer and utility services', 'new york taxis': 'consumer and utility services',
        'air canada': 'consumer and utility services', 'air india': 'consumer and utility services', 'british airways': 'consumer and utility services', 'easyjet': 'consumer and utility services', 'heathrow airport': 'consumer and utility services',
        'buchbinder car rentals': 'consumer and utility services', 'tesla': 'retail and e-commerce', 'toyota': 'retail and e-commerce', 'transport for london': 'consumer and utility services', 'paradise papers': 'government and military',
        'avvo': 'consumer and utility services','benesse': 'media and entertainment','boeing': 'consumer and utility services', 'doordash': 'retail and e-commerce', 'iberdrola': 'consumer and utility services', 'ixigo': 'consumer and utility services', 'nmbs': 'consumer and utility services', 'pandora papers': 'finance and insurance',
        'park mobile': 'technology and telecommunications', 'royal enfield': 'retail and e-commerce', 'star alliance': 'consumer and utility services', 'u-haul': 'consumer and utility services','viewfines': 'technology and telecommunications', 'volkswagen': 'retail and e-commerce',  
    }
    
    for org, sector in replacements.items():
        data.loc[(data['organization'].str.contains(org, case=False, na=False)) & (data['sector'] == 'unknown'), 'sector'] = sector
    return data

cleaned_data['sector'] = cleaned_data['sector'].apply(standardize_sector)
cleaned_data = rename_unknown_sectors(cleaned_data)

#elimino gli unknown senza nome organization e senza settore, non sono utili per l'analisi
cleaned_data = cleaned_data[~((cleaned_data['organization'] == 'unknown') & (cleaned_data['sector'] == 'unknown'))]


**Methods**

In [7]:
def standardize_method(method):
    if isinstance(method, str):
        method = method.lower()
        method_categories = {
            'accidental exposure': ['accidentally exposed', 'accidentally published', 'accidently published', 'accidentally uploaded', 'discs found in trash'],
            'compromised credentials': ['compromised administrative account', 'compromised credentials', 'credential stuffing', 'credential stuffing attack', 'stolen credentials'],
            'cyber attacks including phs&rw': ['brute force attack', 'cyber attack', 'phishing attack', 'ransomware', 'ransomware attack', 'ransomware hacking', 'ransomware/hacked', 'zero-day vulnerabilities'],
            'data misconfiguration': ['data exposed by misconfiguration', 'misconfiguration/poor security', 'misconfigured api', 'misconfigured server', 'misconfigured web server', 'misconfigured website', 'publicly accessible amazon web services (aws) server', 'unprotected api', 'unsecured database', 'unsecured s3 bucket'],
            'data leakage and misuse': ['data leak due to security vulnerabilities', 'data misuse', 'leaked and published', 'compilation of multiple data breaches'],
            'hacking and unauthorized access': ['hacked', 'hacked / poor security', 'hacked and published', 'hacked by alphv', 'hacked by latvian hackers', 'hacked by lulzsec', 'hacked by pwn0001', 'hacked by rhysida', 'hacked by russian hackers', 'hacked by v0g3lsec', 'hacked third party service', 'hacked with a trojan', 'hacked/misconfiguration', 'improper setting, hacked', 'maliciously injected javascript', 'maliciously modified javascript', 'web scraping'],
            'insider threats': ['inside job', 'insider', 'insider theft', 'internal job', 'rogue contractor'],
            'physical theft or loss': ['intentionally lost', 'lost / stolen computer', 'lost / stolen media', 'lost box of data tapes', 'lost device', 'lost/stolen device', 'magnetic tape missing from an iron mountain incorporated storage facility', 'stolen laptop'],
            'security weaknesses': ['poor security', 'poor security / hacked', 'poor security / misconfigured server', 'poor security/account recovery', 'poor security/hacked', 'poor security/inside job', 'vulnerabilities in plugins', 'vulnerability']
        }
        for category, terms in method_categories.items():
            if any(term in method for term in terms):
                return category
    return 'unknown'

cleaned_data['method'] = cleaned_data['method'].apply(standardize_method)

**Duplicates**

In [ ]:
def is_numeric_value(value):
    if isinstance(value, str) and value.isdigit():
        return True
    return False

cleaned_data['numeric_records_lost'] = cleaned_data['records lost'].apply(is_numeric_value)
cleaned_data = cleaned_data.sort_values(by=['organization', 'records lost', 'year', 'numeric_records_lost'], 
                                        ascending=[True, True, True, False])

cleaned_data = cleaned_data.drop_duplicates(subset=['organization', 'records lost'], keep='first')
cleaned_data = cleaned_data.drop(columns=['numeric_records_lost'])

**Sorted and save new cleaned dataset**

In [9]:
final_filtered_data_sorted = cleaned_data.sort_values(by='organization', ignore_index=True)
final_filtered_data_sorted.to_csv("final_data_breaches.csv", index=False)